In [1]:
import numpy as np
#import scipy.misc
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline

import math
import random

#from random import shuffle

In [2]:
X = [1,2,3,4,5]
random.shuffle(X)
X

[3, 1, 2, 5, 4]

In [25]:
numNetworks = 100
numNodesPerNetwork = 100
inputFeatures = 10
outputFeatures = 5

globalInnovationNumber = 0

assert inputFeatures < numNodesPerNetwork
assert outputFeatures < numNodesPerNetwork

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Node:
    """
    This represents a neural network node with an integer label nodeId, 
    and a special type, one of ["input", "output", "hidden"]
    """
    def __init__(self, nodeId, nodeType):
        self.nodeId = nodeId
        assert nodeType == "input" or nodeType == "output" or nodeType == "hidden"
        self.nodeType = nodeType
        
    def copy(self):
        return Node(self.nodeId, self.nodeType)    
        
    def __str__(self):
        return "Node: " + str(self.nodeId) + ", " + self.nodeType
    
        
class Edge:
    """
    This class represent a connection (axon) in a neuroevolution network with its data.
    nodeIn and nodeOut are the ids of the input and output nodes,
    enabled is a boolean representing its ability to propogate into future neural networks,
    innovId is the labeled innovation number which corresponds to the same edge in other NNs,
    and weight is the normal edge weight.
    """
    def __init__(self, nodeIn, nodeOut, enabled, innovId, weight):
        self.nodeInId = nodeIn
        self.nodeOutId = nodeOut
        self.enabled = enabled
        self.innovId = innovId
        self.weight = weight
        
    def copy(self):
        return Edge(self.nodeInId, self.nodeOutId, self.enabled, self.innovId, self.weight)
        
    def __str__(self):
        return "Edge: " + str(self.nodeInId) + " -> " + str(self.nodeOutId) + ", Enabled: " + \
            str(self.enabled) + ", InnovID: " + str(self.innovId) + ", Weight: " + str(self.weight)
    
class NeuroevolutionNetwork:
    
    # self.nodes is a dictionary indexed by (node id -> node object)
    # self.edges is indexed by (in node id -> edge object, from in node to out node)
    def __init__(self, numNodes, numInput, numOutput):
        assert numInput + numOutput < numNodes
        
        self.nodes = dict() 
        self.edges = dict()
        self.listEdges = []
        self.numNodesCounter = 0
        
        self.numInput = numInput
        self.numOutput = numOutput
        self.initNodes = numNodes
        
        for _ in range(numInput):
            self.addNode("input")
        for _ in range(numOutput):
            self.addNode("output")
        for _ in range(numNodes - numInput - numOutput):
            self.addNode("hidden")
        
    # Special copy method to handle it ourselves, so it's not just a shallow/deep clone
    def copy(self):
        copyNetwork = NeuroevolutionNetwork(self.initNodes, self.numInput, self.numOutput)
        
        for nodeId, node in self.nodes.items():
            # For every node id, copy its node to the new network
            copyNetwork.nodes[nodeId] = node.copy()
            
        for nodeId, edgeList in self.edges.items():
            # For every node in id, copy the list of outgoing edges to the new network
            copyEdgeArr = []
            for edge in edgeList:
                copyEdgeArr.append(edge.copy())
            copyNetwork.edges[nodeId] = copyEdgeArr  
            
        for pair in self.listEdges:
            copyNetwork.listEdges.append(tuple(pair))
            
        return copyNetwork
    
    def printAllEdges(self):
        for nodeInId, nodeArr in self.edges.items():
            print(nodeInId)
            for node in nodeArr:
                print(str(node))
                
    def numNodes(self):
        return len(self.nodes)
    
    def numEdges(self):
        return len(self.listEdges)
        
    def addNode(self, nodeType):
        nodeId = self.numNodesCounter
        self.nodes[nodeId] = Node(nodeId, nodeType)
        self.numNodesCounter += 1
        return nodeId
        
    def hasNode(self, nodeId):
        return nodeId in self.nodes    
        
    # Return true if a connection exists between nodes with these ids,
    # a connection from in -> out
    def isConnected(self, nodeInId, nodeOutId):    
        """if hasNode(nodeInId) and hasNode(nodeOutId):
            if nodeInId in self.edges:
                connectionsOut = self.edges[nodeInId]
                for nodeOut in connectionsOut:
                    if nodeOut.nodeId == nodeOutId:
                        return True
        return False"""
        return (nodeInId, nodeOutId) in self.listEdges
    
    # Return true if there is a valid connection from in -> out
    # Invariant to whether or not the nodes are connected or not
    def allowedConnection(self, nodeInId, nodeOutId):
        if self.hasNode(nodeInId) and self.hasNode(nodeOutId):
            inNode = self.nodes[nodeInId]
            outNode = self.nodes[nodeOutId]
            doubleInput = inNode.nodeType == "input" and outNode.nodeType == "output"
            doubleOutput = inNode.nodeType == "input" and outNode.nodeType == "output"
            return not doubleInput and not doubleOutput
        else:
            return False
        
    # Variant to whether nodes are connected. To be able to add a new connection, the nodes
    # must follow these conditions:
    """
    Nodes exist
    Nodes are not connected in either direction
    The end node is not an input
    The start node is not an output
    New: the new edge cannot create a cycle
    """
    def canAddConnection(self, nodeInId, nodeOutId):
        assert nodeInId != nodeOutId
        if not self.allowedConnection(nodeInId, nodeOutId):
            return False
        inToOutC = self.isConnected(nodeInId, nodeOutId)
        outToInC = self.isConnected(nodeOutId, nodeInId)
        endInput = self.nodes[nodeOutId].nodeType == "input"
        startOutput = self.nodes[nodeInId].nodeType == "output"
        longPathExists = self.findPathBetween(nodeOutId, nodeInId) # Check for possible cycles
        return not (inToOutC or outToInC or endInput or startOutput or longPathExists)
        
    # Select a random pair of nodes, that have no connection either way,
    # and are not both input and not both output,
    # and add a connection between them, from in -> out.
    def mutateAddConnection(self):
        assert len(self.nodes) >= 2
        
        # Generate a shuffled list of possible future connections
        availIds = [node.nodeId for _,node in self.nodes.items()]
        lenIds = len(availIds)
        availablePairs = [(availIds[i], availIds[j]) for i in range(lenIds) for j in range(lenIds)]
        random.shuffle(availablePairs)
        
        # Find two nodes that have no connection
        # Once found, add the one new connection, and return
        while True: 
            if len(availablePairs) == 0: # No more possible connections to check
                return
            pair = availablePairs.pop() 
            if pair[0] == pair[1]:
                continue
            # Select a random connection, that is neither a self connection (does not exist),
            # nor an existing connection in either direction.
            # inToOutC = self.isConnected(pair[0], pair[1])
            # outToInC = self.isConnected(pair[1], pair[0])
            if self.canAddConnection(pair[0], pair[1]):
                global globalInnovationNumber
                self.addEdgeWithInnovId(pair[0], pair[1], True, globalInnovationNumber, edgeWeight=0.5)
                globalInnovationNumber += 1
                return
            
    def findEdge(self, nodeInId, nodeOutId):
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            possibleEdge = self.edges[nodeInId][nodeOutIndex]
            if possibleEdge.nodeOutId == nodeOutId:
                return possibleEdge
        return None
        
    def removeEdge(self, nodeInId, nodeOutId):
        assert nodeInId in self.edges
        assert (nodeInId, nodeOutId) in self.listEdges
        
        self.listEdges.remove((nodeInId, nodeOutId))
        
        # Look for the correct node within the outgoing connections
        # i.e. find the connection in -> out
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            nodeOut = self.edges[nodeInId][nodeOutIndex]
            if nodeOut.nodeOutId == nodeOutId:
                self.edges[nodeInId].pop(nodeOutIndex)
                return  
            
    # Simple tree traversal, to check for a path between in -> out, of any length
    def findPathBetween(self, nodeInId, nodeOutId):
        fringe = [nodeInId]
        marked = set()
        while len(fringe) > 0:
            v = fringe.pop(0)
            if v == nodeOutId: # This case is if nodeInId == nodeOutId
                return True
            marked.add(v)
            if v not in self.edges: # No edges starting from v
                continue
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if w == nodeOutId:
                    return True
                if w not in marked:
                    fringe.append(w)
        return False
            
    def addEdgeWithInnovId(self, nodeInId, nodeOutId, enabled, innovId, edgeWeight):
        assert self.canAddConnection(nodeInId, nodeOutId)
        
        newEdge = Edge(nodeInId, nodeOutId, enabled, innovId, edgeWeight)
        
        self.listEdges.append((nodeInId, nodeOutId))
        if nodeInId not in self.edges:
            self.edges[nodeInId] = []
        self.edges[nodeInId].append(newEdge)
            
    """
    This mutates the graph by finding a random existing edge,
    and then splitting into two edges with a new connection in the middle,
    as per Stanley & Miikkulainen.
    """    
    def mutateSplitConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        randomEdgeIndex = int(len(self.listEdges) * random.random())
        oldEdge = tuple(self.listEdges[randomEdgeIndex])
        nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
        
        # TEMP:
        self.removeEdge(nodeInId, nodeOutId)
        # No, do not remove the edge. It must be preserved to play a part in later
        # NN "reproduction" and similarity measures
        
        newMiddleNode = self.addNode("hidden")
        
        global globalInnovationNumber
        self.addEdgeWithInnovId(nodeInId, newMiddleNode, True, globalInnovationNumber, edgeWeight=0.5)
        globalInnovationNumber += 1
        self.addEdgeWithInnovId(newMiddleNode, nodeOutId, True, globalInnovationNumber, edgeWeight=0.5)
        globalInnovationNumber += 1
        
    """
    Get all of the directed edges of the NN, but in reverse i.e. incoming edges,
    also representing an in-degree count indexed by vertex.
    """
    def getReversedEdges(self):
        reversedEdges = dict()
        
        for edgeInId, edgeList in self.edges.items():
            for edge in edgeList:
                if edge.nodeOutId not in reversedEdges:
                    reversedEdges[edge.nodeOutId] = []
                reversedEdges[edge.nodeOutId].append(edge.nodeInId)
        
        return reversedEdges
    
    """
    The new forwarding algorithm works as follows, in a DAG:
    obtain the edges of the graph, reversed;
    determine the in-degree of each vertex, using above;
    use Kahn's algorithm -> remove vertices of in-degree zero,
    repeat until end or a valid topological sort has been found;
    update vertices in topological order
    
    Proof that Kahn's algorithm terminates in a valid topo sort <-> graph is a DAG
    is left as an exercise for the reader.
    """
    
    """
    Return a list of the nodes 
    that represent a valid topological sort,
    starting with input edges (guaranteed to be in-degree zero).
    """
    def graphTopoSort(self):
        reversedEdges = self.getReversedEdges()
        
        marked = dict()
        topoSort = []
        
        for nodeId,_ in self.nodes.items():
            marked[nodeId] = False
        
        while len(topoSort) < len(self.nodes):
            degZeroIncomingIds = []
            
            for nodeId,_ in self.nodes.items():
                # Boolean short circ. V
                # The second case can only come about when connections are discarded
                if (nodeId not in reversedEdges or len(reversedEdges[nodeId]) == 0) and not marked[nodeId]: 
                    # nodeId must have an in-degree zero
                    degZeroIncomingIds.append(nodeId)
                    marked[nodeId] = True
                    
                    # No outgoing edges, we're done
                    if nodeId not in self.edges:
                        continue
                    
                    # Discard all outgoing connections
                    outgoingEdges = self.edges[nodeId]
                    
                    for edge in outgoingEdges:
                        if edge.nodeOutId in reversedEdges:
                            reversedEdges[edge.nodeOutId].remove(nodeId)
            
            topoSort = topoSort + degZeroIncomingIds
        return topoSort
    
    """
    Calculate the entire NN output, starting from the input.
    Again, the topological ordering is used to guarantee the nodes are calculated correctly
    i.e. all incoming edges have valid values.
    """
    def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        # nodeData is the final result of the NN, indexed by nodeId
        # Initialize the input, which is ideally of shape (input nodes X 1)
        nodeData = dict()
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
        
        reversedEdges = self.getReversedEdges()
        topoSort = self.graphTopoSort()
        
        for nodeId in topoSort:
            nodeType = self.nodes[nodeId].nodeType
            if nodeType == 'input':
                continue
            
            incomingEdges = []
            if nodeId in reversedEdges:
                incomingEdges = reversedEdges[nodeId]
            
            result = 0
            if len(incomingEdges) == 0:
                result = 0.5
            else:
                nodeSum = 0
                for inVertex in incomingEdges:
                    value = nodeData[inVertex]
                    weight = self.findEdge(inVertex, nodeId).weight
                    nodeSum += value * weight
                result = sigmoid(nodeSum)
                
            nodeData[nodeId] = result
        
        return nodeData
    
    def score(self, currentNodeData, desiredOutput, lossFunction):
        predictedOutput = np.zeros((self.numOutput))
        for i in range(0, self.numOutput):
            predictedOutput[i] = currentNodeData[self.numInput + i]
        assert predictedOutput.shape == desiredOutput.shape
        loss = lossFunction(predictedOutput, desiredOutput)
        return predictedOutput, desiredOutput, loss
        
    """def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
            
        # Use Dijkstra's algorithm to update the NN as we go, 
        # using the input data as the starting fringe.
        
        # Ideally, the NN is a DAG, but this will work even if not.
        
        nodeData = dict()
        prev = dict()
        marked = dict()
        
        queue = [i for i in range(len(self.numInput))]
        
        for nodeId, _ in self.nodes.items():
            nodeData[nodeId] = -9999
            prev[nodeId] = -1
            marked[nodeId] = False
        
        while len(queue) > 0:
            v = queue.pop() # A node id
            marked[v] = True
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if :
                    nodeData[]"""

In [26]:
# Unit tests for neuroevolution network

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)

# Nodes are initialized correctly
"""
for k,v in testNet.nodes.items():
    print(v)
"""
    
# Edges are initialized correctly
print("-----------------------------------")
print("Test #2")
print(testNet.edges, testNet.listEdges)

print("-----------------------------------")
print("Test #3")
for _ in range(5):
    testNet.mutateAddConnection()

testNet.printAllEdges()
    
print(testNet.listEdges)

print("-----------------------------------")
print("Test #4")

testEdge = testNet.listEdges[0]
testNet.removeEdge(testEdge[0], testEdge[1])

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5")

testNet.mutateSplitConnection()

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #6")

testNetCopy = testNet.copy()
testNetCopy.mutateAddConnection()

print("Original network: ")
testNet.printAllEdges()
print("Copy network: ")
testNetCopy.printAllEdges()

-----------------------------------
Test #2
{} []
-----------------------------------
Test #3
53
Edge: 53 -> 39, Enabled: True, InnovID: 0, Weight: 0.5
17
Edge: 17 -> 98, Enabled: True, InnovID: 1, Weight: 0.5
Edge: 17 -> 16, Enabled: True, InnovID: 2, Weight: 0.5
4
Edge: 4 -> 96, Enabled: True, InnovID: 3, Weight: 0.5
3
Edge: 3 -> 99, Enabled: True, InnovID: 4, Weight: 0.5
[(53, 39), (17, 98), (17, 16), (4, 96), (3, 99)]
-----------------------------------
Test #4
53
17
Edge: 17 -> 98, Enabled: True, InnovID: 1, Weight: 0.5
Edge: 17 -> 16, Enabled: True, InnovID: 2, Weight: 0.5
4
Edge: 4 -> 96, Enabled: True, InnovID: 3, Weight: 0.5
3
Edge: 3 -> 99, Enabled: True, InnovID: 4, Weight: 0.5
[(17, 98), (17, 16), (4, 96), (3, 99)]
-----------------------------------
Test #5
53
17
Edge: 17 -> 16, Enabled: True, InnovID: 2, Weight: 0.5
Edge: 17 -> 100, Enabled: True, InnovID: 5, Weight: 0.5
4
Edge: 4 -> 96, Enabled: True, InnovID: 3, Weight: 0.5
3
Edge: 3 -> 99, Enabled: True, InnovID: 4, We

In [27]:
testNode = Node(85, 'hidden')
print(testNode.copy())

Node: 85, hidden


In [34]:
"""
Calculate the distance delta between two neuroevolution networks.
This heuristic is important in determining a normalized fitness score,
which rewards successful organisms but ensures that one species does not take over.

In Stanley, Miikkulainen, this is defined as

delta = c_1 * E / N + c_2 * D / N + c_3 * W
where E is the number of excess genes,
D is the number of disjoint genes, 
N is the normalization factor (number of genes total),
and W is the average weight distance between shared genes.

Ideally, the weights have constraint 0 <= c_1, c_2, c_3 <= 1, 
and should also be consistent.
"""
def neuroNetworkDiff(nn1, nn2, c1, c2, c3):
    # Calculate excess number of genes by count
    excess = abs(nn1.numEdges() - nn2.numEdges())
    
    # Disjoint genes just need to be counted,
    # shared genes must have their weight differences averaged
    disjoint = 0
    sharedGenesDiff = []
    
    numGenes = max(nn1.numEdges(), nn2.numEdges())
    
    firstEdgeInnovIds = set()
    weightsByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            weightsByInnovId[edge.innovId] = edge.weight
    
    for _,outEdges in nn2.edges.items():
        for secondEdge in outEdges:
            if secondEdge.innovId in firstEdgeInnovIds:
                firstEdgeWeight = weightsByInnovId[secondEdge.innovId]
                secondEdgeWeight = secondEdge.weight
                # secondEdge = nn2.edges[node.nodeId]
                sharedGenesDiff.append(abs(firstEdgeWeight - secondEdgeWeight))
            else:
                disjoint += 1

    avgDiffW = np.sum(sharedGenesDiff) / len(sharedGenesDiff)
    
    print(excess, disjoint, numGenes, avgDiffW)
    
    return c1 * excess / numGenes + \
        c2 * disjoint / numGenes + \
        c3 * avgDiffW
        

def crossNeuralNetworks(nn1, nn2):
    assert nn1.numInput == nn2.numInput
    assert nn1.numOutput == nn2.numOutput
    newNumNodes = max(nn1.numNodes(), nn2.numNodes())
    
    crossedNet = NeuroevolutionNetwork(newNumNodes, nn1.numInput, nn1.numOutput)
    
    firstEdgeInnovIds = set()
    secondEdgeInnovIds = set()
    firstEdgesByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    secondEdgesByInnovId = dict()
    enabledByInnovId = dict()
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            firstEdgesByInnovId[edge.innovId] = edge
            enabledByInnovId[edge.innovId] = edge.enabled
    
    for _,outEdges in nn2.edges.items():
        for edge in outEdges:
            secondEdgeInnovIds.add(edge.innovId)
            secondEdgesByInnovId[edge.innovId] = edge
            if edge.innovId in enabledByInnovId:
                enabledByInnovId[edge.innovId] = enabledByInnovId[edge.innovId] and edge.enabled
            else:
                enabledByInnovId[edge.innovId] = edge.enabled
    
    """
    As per Stanley and Miikkulainen, merge the edges together by their innovation ids.
    Disjoint/excess innovations transfer fully, shared innovations must merge,
    weight by average, enabled by AND
    """
    
    sharedInnov = firstEdgeInnovIds.intersection(secondEdgeInnovIds)
    onlyFirstInnov = firstEdgeInnovIds.difference(secondEdgeInnovIds)
    onlySecondInnov = secondEdgeInnovIds.difference(firstEdgeInnovIds)
    
    #nodeIn, nodeOut, enabled, innovId, weight)
    for innovId in onlyFirstInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
        
    for innovId in onlySecondInnov:
        baseEdge = secondEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
            
    for innovId in sharedInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        
        firstWeight = firstEdgesByInnovId[innovId].weight
        secondWeight = secondEdgesByInnovId[innovId].weight
        avgWeight = 0.5 * (firstWeight + secondWeight)
        
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, avgWeight)
            
    print(sharedInnov)
    
    return crossedNet
                

def squaredLoss(predicted, desired):
    return np.sum((predicted - desired) ** 2)    



In [35]:
print("-----------------------------------")
print("Test #7")

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)
for _ in range(10):
    testNet.mutateAddConnection()
    
testNetMutate = testNet.copy()

for _ in range(2):
    testNetMutate.mutateAddConnection()
for _ in range(5):
    testNetMutate.mutateSplitConnection()
    
testNet.printAllEdges()
testNetMutate.printAllEdges()

print(neuroNetworkDiff(testNet, testNetMutate, 0.5, 0.25, 0.2))

print("-----------------------------------")
print("Test #8")

testEdges = testNet.getReversedEdges()
for nodeId,listEdges in testEdges.items():
    print(nodeId)
    print(listEdges)

print("-----------------------------------")
print("Test #9")

testNet.printAllEdges()
topoSort = testNet.graphTopoSort()
print(topoSort)

-----------------------------------
Test #7
56
Edge: 56 -> 76, Enabled: True, InnovID: 99, Weight: 0.5
90
Edge: 90 -> 78, Enabled: True, InnovID: 100, Weight: 0.5
92
Edge: 92 -> 15, Enabled: True, InnovID: 101, Weight: 0.5
Edge: 92 -> 30, Enabled: True, InnovID: 108, Weight: 0.5
4
Edge: 4 -> 20, Enabled: True, InnovID: 102, Weight: 0.5
97
Edge: 97 -> 13, Enabled: True, InnovID: 103, Weight: 0.5
88
Edge: 88 -> 41, Enabled: True, InnovID: 104, Weight: 0.5
87
Edge: 87 -> 29, Enabled: True, InnovID: 105, Weight: 0.5
52
Edge: 52 -> 34, Enabled: True, InnovID: 106, Weight: 0.5
82
Edge: 82 -> 31, Enabled: True, InnovID: 107, Weight: 0.5
56
Edge: 56 -> 76, Enabled: True, InnovID: 99, Weight: 0.5
90
Edge: 90 -> 78, Enabled: True, InnovID: 100, Weight: 0.5
92
Edge: 92 -> 30, Enabled: True, InnovID: 108, Weight: 0.5
Edge: 92 -> 100, Enabled: True, InnovID: 111, Weight: 0.5
4
Edge: 4 -> 103, Enabled: True, InnovID: 117, Weight: 0.5
97
Edge: 97 -> 104, Enabled: True, InnovID: 119, Weight: 0.5
88
Ed

In [36]:
print("-----------------------------------")
print("Test #10")

testNet = NeuroevolutionNetwork(15, 5, 5)
for _ in range(20):
    testNet.mutateAddConnection()
for _ in range(3):
    testNet.mutateSplitConnection()
for _ in range(20):
    testNet.mutateAddConnection()
    
testNet.printAllEdges()

-----------------------------------
Test #10
0
Edge: 0 -> 13, Enabled: True, InnovID: 121, Weight: 0.5
Edge: 0 -> 10, Enabled: True, InnovID: 138, Weight: 0.5
Edge: 0 -> 14, Enabled: True, InnovID: 149, Weight: 0.5
14
Edge: 14 -> 11, Enabled: True, InnovID: 122, Weight: 0.5
Edge: 14 -> 10, Enabled: True, InnovID: 129, Weight: 0.5
Edge: 14 -> 9, Enabled: True, InnovID: 131, Weight: 0.5
Edge: 14 -> 15, Enabled: True, InnovID: 141, Weight: 0.5
Edge: 14 -> 16, Enabled: True, InnovID: 164, Weight: 0.5
10
Edge: 10 -> 5, Enabled: True, InnovID: 123, Weight: 0.5
Edge: 10 -> 9, Enabled: True, InnovID: 132, Weight: 0.5
Edge: 10 -> 6, Enabled: True, InnovID: 135, Weight: 0.5
Edge: 10 -> 8, Enabled: True, InnovID: 136, Weight: 0.5
Edge: 10 -> 11, Enabled: True, InnovID: 139, Weight: 0.5
3
Edge: 3 -> 14, Enabled: True, InnovID: 124, Weight: 0.5
Edge: 3 -> 10, Enabled: True, InnovID: 165, Weight: 0.5
1
Edge: 1 -> 14, Enabled: True, InnovID: 125, Weight: 0.5
4
Edge: 4 -> 13, Enabled: True, InnovID: 1

In [37]:
testNet.forwardStep([1.0, 0.0, 1.0, 0.0, 1.0])
# [16, 10, 4, 1, 97]
# -> [4,3.2,2,1,9.7]

{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.75137444050395341,
 6: 0.76814869688571041,
 7: 0.81690904094907846,
 8: 0.83076530280318461,
 9: 0.70731739176200314,
 10: 0.87489424500596802,
 11: 0.77241181701633899,
 12: 0.5,
 13: 0.83701801338742643,
 14: 0.88988719327660015,
 15: 0.79103071284792981,
 16: 0.74846813560890457,
 17: 0.67917869917539297}

In [38]:
print("-----------------------------------")
print("Test #11")

# score(self, currentNodeData, desiredOutput, lossFunction):

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
totalNodeData = testNet.forwardStep(inputData)
desired = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

predicted, _, testLoss = testNet.score(totalNodeData, desired, squaredLoss)
print(predicted)
print(desired)
print(testLoss)

-----------------------------------
Test #11
[ 0.75137444  0.7681487   0.81690904  0.8307653   0.70731739]
[ 0.  1.  0.  1.  0.]
1.81459723321


In [39]:
print("-----------------------------------")
print("Test #12")

testNet = NeuroevolutionNetwork(15, 5, 5)

for _ in range(5):
    testNet.mutateAddConnection()
    
testNetMutate1 = testNet.copy()
testNetMutate2 = testNet.copy()

for _ in range(5):
    testNetMutate1.mutateAddConnection()
for _ in range(2):
    testNetMutate1.mutateSplitConnection()

for _ in range(5):
    testNetMutate2.mutateAddConnection()
for _ in range(2):
    testNetMutate2.mutateSplitConnection()
    
print("**************************")
testNet.printAllEdges()
print("**************************")
testNetMutate1.printAllEdges()
print("**************************")
testNetMutate2.printAllEdges()
print("**************************")

crossedNet = crossNeuralNetworks(testNetMutate1, testNetMutate2)
crossedNet.printAllEdges()

-----------------------------------
Test #12
**************************
1
Edge: 1 -> 13, Enabled: True, InnovID: 167, Weight: 0.5
3
Edge: 3 -> 11, Enabled: True, InnovID: 168, Weight: 0.5
14
Edge: 14 -> 11, Enabled: True, InnovID: 169, Weight: 0.5
11
Edge: 11 -> 9, Enabled: True, InnovID: 170, Weight: 0.5
Edge: 11 -> 13, Enabled: True, InnovID: 171, Weight: 0.5
**************************
1
Edge: 1 -> 13, Enabled: True, InnovID: 167, Weight: 0.5
3
Edge: 3 -> 11, Enabled: True, InnovID: 168, Weight: 0.5
Edge: 3 -> 10, Enabled: True, InnovID: 174, Weight: 0.5
14
Edge: 14 -> 11, Enabled: True, InnovID: 169, Weight: 0.5
Edge: 14 -> 5, Enabled: True, InnovID: 175, Weight: 0.5
11
Edge: 11 -> 9, Enabled: True, InnovID: 170, Weight: 0.5
Edge: 11 -> 16, Enabled: True, InnovID: 179, Weight: 0.5
12
Edge: 12 -> 5, Enabled: True, InnovID: 172, Weight: 0.5
13
Edge: 13 -> 12, Enabled: True, InnovID: 173, Weight: 0.5
2
Edge: 2 -> 15, Enabled: True, InnovID: 177, Weight: 0.5
15
Edge: 15 -> 11, Enabled: 